In [ ]:
from openai import OpenAI

# filepath: /path/to/your/file.py
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Get the OpenAI API key
openai_api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI()

In [4]:


client.files.create(
  file=open("dad_joke_samples_train.jsonl", "rb"),
  purpose="fine-tune"
)

client.files.create(
  file=open("dad_joke_samples_test.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-14gtA38iuJ7sJgqBmwsyNC', bytes=4770, created_at=1735328489, filename='dad_joke_samples_test.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [14]:
files = client.files.list(purpose="fine-tune")

for file in files:
    print(file.filename.upper(),':',file.id)

DAD_JOKE_SAMPLES_TEST.JSONL : file-14gtA38iuJ7sJgqBmwsyNC
DAD_JOKE_SAMPLES_TRAIN.JSONL : file-Jp47nPA2Eq2S9p7VXCEsWz


In [15]:
client.fine_tuning.jobs.create(
  training_file="file-Jp47nPA2Eq2S9p7VXCEsWz",
  validation_file ="file-14gtA38iuJ7sJgqBmwsyNC",
  model="gpt-4o-mini-2024-07-18"
)

FineTuningJob(id='ftjob-34rI6p35wsPEGLjrK6R1pUCp', created_at=1735328907, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-RuBNDg5GaMqiVIMAjwK7becX', result_files=[], seed=492436941, status='validating_files', trained_tokens=None, training_file='file-Jp47nPA2Eq2S9p7VXCEsWz', validation_file='file-14gtA38iuJ7sJgqBmwsyNC', estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto')), type='supervised'), user_provided_suffix=None)

In [39]:
import time


while True:
    events = client.fine_tuning.jobs.list_events(fine_tuning_job_id="ftjob-34rI6p35wsPEGLjrK6R1pUCp", limit=10)
    message = events.data[0].message
    print(message)
    if message == "The job has successfully completed":
        break
    

    time.sleep(20)

The job has successfully completed


In [56]:
# Retrieve the state of a fine-tune
dad_joke_model = client.fine_tuning.jobs.retrieve("ftjob-34rI6p35wsPEGLjrK6R1pUCp").fine_tuned_model
print(dad_joke_model)

ft:gpt-4o-mini-2024-07-18:personal::AjAj8dcn


In [55]:
def get_response(messages):
    response = client.chat.completions.create(
        model=dad_joke_model,
        messages=messages
    )
    return response.choices[0].message.content

def main():
    print("Welcome to the Dad Joke Chatbot! Type 'exit' to quit.")
    messages = [{"role": "system", "content": "You are a dad joke expert. Create dad jokes in accordance with the user input."}]
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            break
        messages.append({"role": "user", "content": user_input})
        response = get_response(messages=messages)
        messages.append({"role": "assistant", "content": response})
        print(f"Dad Joke Bot: {response}")


if __name__ == "__main__":
    main()

Welcome to the Dad Joke Chatbot! Type 'exit' to quit.
Dad Joke Bot: Bir baba çocuğuna düğünde herkesin yaptığı dansa ne denir? diye sormuş.
Dad Joke Bot: Gelin Hayatından Emin Midir.
